# Code examples

In [1]:
try:
    from main_estimate import *
except:
    print('Error check packages dependencies / Kernel')

## CPIframe

```python
# === Data (index = HICP & Qt without log transf.) ===
df_w = pd.read_excel("data/data_flat.xlsx",sheet_name="weights")
df_q_index = pd.read_excel("data/data_flat.xlsx",sheet_name="QT_index")
df_p_index = pd.read_excel("data/data_flat.xlsx",sheet_name="P_index")

class CPIframe:
    def __init__(self, df_q_index, df_p_index, df_w, country):
        """
        Args:
            df_q_index (DataFrame): The Quantity data (index).
            df_p_index (DataFrame): The Price data (index).
            df_w (DataFrame): The respective Weights data.
            country (str): Location available (EU27,France,Germany,Spain).
        """
````

In [2]:
eu = CPIframe(df_q_index=df_q_index, df_p_index=df_p_index, df_w=df_w, country="EU27")
print([atr for atr in dir(eu) if "__" not in atr])

['country', 'dates', 'flag_sectors', 'framing', 'inflation', 'price', 'price_index', 'qt', 'qt_index', 'sector', 'sectors', 'weights']


- *flag_sectors*, *framing* and *sector* are methods for the class *CPIframe*
- *price_index* and *price* (data of *price_index* log-transformed) are subset dataframes of initial *df_p_index* (Price data) restricted to *country*
- *qt_index* and *qt* (data of *qt_index* log-transformed) are subset dataframes of initial *df_q_index* (Quantity data = demand proxy) restricted to *country*

In [3]:
eu.sectors   #COICOP 4-digit categories
#eu.inflation #MoM inflation rate dataframe
#eu.weights    #DataFrame of Annual sector weights in the overall HICP 
#eu.price
#eu.price_index
#eu.qt
#eu.qt_index

{0: 'Bread and cereals',
 1: 'Meat',
 2: 'Fish and seafood',
 3: 'Milk, cheese and eggs',
 4: 'Oils and fats',
 5: 'Fruit',
 6: 'Vegetables',
 7: 'Sugar, jam, honey, chocolate and confectionery',
 8: 'Food products n.e.c.',
 9: 'Coffee, tea and cocoa',
 10: 'Mineral waters, soft drinks, fruit and vegetable juices',
 11: 'Spirits',
 12: 'Wine',
 13: 'Beer',
 14: 'Tobacco',
 15: 'Clothing materials',
 16: 'Garments',
 17: 'Other articles of clothing and clothing accessories',
 18: 'Cleaning, repair and hire of clothing',
 19: 'Shoes and other footwear',
 20: 'Repair and hire of footwear',
 21: 'Actual rentals for housing',
 22: 'Maintenance and repair of the dwelling',
 23: 'Water supply and miscellaneous services relating to the dwelling',
 24: 'Electricity, gas and other fuels',
 25: 'Furniture and furnishings',
 26: 'Carpets and other floor coverings',
 27: 'Repair of furniture, furnishings and floor coverings',
 28: 'Household textiles',
 29: 'Major household appliances whether elect

In [4]:
eu.flag_sectors()

0  : OK  2001-01 ; 2023-09  --  273  obs. --  Bread and cereals
1  : OK  2001-01 ; 2023-09  --  273  obs. --  Meat
2  : OK  2001-01 ; 2023-09  --  273  obs. --  Fish and seafood
3  : OK  2001-01 ; 2023-09  --  273  obs. --  Milk, cheese and eggs
4  : OK  2001-01 ; 2023-09  --  273  obs. --  Oils and fats
5  : OK  2001-01 ; 2023-09  --  273  obs. --  Fruit
6  : OK  2001-01 ; 2023-09  --  273  obs. --  Vegetables
7  : OK  2001-01 ; 2023-09  --  273  obs. --  Sugar, jam, honey, chocolate and confectionery
8  : OK  2001-01 ; 2023-09  --  273  obs. --  Food products n.e.c.
9  : OK  2001-01 ; 2023-09  --  273  obs. --  Coffee, tea and cocoa
10  : OK  2001-01 ; 2023-09  --  273  obs. --  Mineral waters, soft drinks, fruit and vegetable juices
11  : OK  2001-01 ; 2023-09  --  273  obs. --  Spirits
12  : OK  2001-01 ; 2023-09  --  273  obs. --  Wine
13  : OK  2001-01 ; 2023-09  --  273  obs. --  Beer
14  : OK  2001-01 ; 2023-09  --  273  obs. --  Tobacco
15  : end missing - 2001-01 ; 2022-04  -

## sector_estimation

Builds on a `CPIframe`object.

```python
class sector_estimation:
    def __init__(self,
                 meta:CPIframe,
                 col:int,
                 order:Union[int, str]="auto",maxlag=24,trend="n",
                 shapiro:bool=True,
                 shapiro_robust:bool=False,
                 sheremirov:bool=True,
                 sheremirov_window:list=[1,11]):
        """
        Args:
            meta: CPIframe object
            col: sector column number in [0,93]
                => If too litte data for sector 'col' then raises ValueError
                => VAR model is built with first diff then demeand log-transformed data
            `VAR parametrization`
            order: - If "auto" VAR order is automatically selected
                   - Else enter an integer
            maxlag: higher bound of order selection
            trend: should remain "n" because data have been demeaned and are supposed to be stationary (no trend)
            `Labeling methods`
            shapiro: if True computes baseline Shapiro(2022) labeling method with estimated VAR
            shapiro_robustness: if True also computes alternative labeling methodologies
            sheremirov: if True computes baseline Sheremirov(2022) labeling method
            sheremirov_window: [Transitory,Persistent] parametrization of step classification algo step 5.
        """

**Automatic lag order selection**

In [13]:
test_sect_estim1 = sector_estimation(meta=eu, col=64, shapiro_robust=True)
# When order is automatically selected results accessible via aic and bic are accessed via (...).aic or (...).bic

In [8]:
#test_sect_estim1.aic.summary() #Estimated model with AIC
#test_sect_estim1.aic.shapiro 
display(test_sect_estim1.aic.shapiro_robust)
display(test_sect_estim1.sheremirov)

,dem,temp,dem_j1,sup_j1,dem_j2,sup_j2,dem_j3,sup_j3
2000-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-03,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-04,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-05,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...
2023-05,1,0,1,0,0,1,0,1
2023-06,0,1,1,0,1,0,1,0
2023-07,0,1,1,0,1,0,0,1
2023-08,0,1,0,1,0,1,1,0


,dem,sup,dem_pers,dem_trans,sup_pers,sup_trans
2000-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-03,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-04,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-05,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...
2023-05,0,1,0,0,1,0
2023-06,0,1,0,0,1,0
2023-07,0,1,0,0,1,0
2023-08,0,1,0,0,1,0


**Selected order**

In [14]:
test_sect_estim2 = sector_estimation(meta=eu, col=64, order=12, shapiro_robust=True)
# When order is hand-fixed results are accessible via (...).estimate

In [16]:
#test_sect_estim2.estimate.summary() #Estimated model with selected order
#test_sect_estim2.estimate.shapiro 
display(test_sect_estim1.aic.shapiro_robust)
display(test_sect_estim1.sheremirov)

,dem,temp,dem_j1,sup_j1,dem_j2,sup_j2,dem_j3,sup_j3
2000-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-03,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-04,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-05,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...
2023-05,1,0,1,0,0,1,0,1
2023-06,0,1,1,0,1,0,1,0
2023-07,0,1,1,0,1,0,0,1
2023-08,0,1,0,1,0,1,1,0


,dem,sup,dem_pers,dem_trans,sup_pers,sup_trans
2000-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-03,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-04,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-05,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...
2023-05,0,1,0,0,1,0
2023-06,0,1,0,0,1,0
2023-07,0,1,0,0,1,0
2023-08,0,1,0,0,1,0
